<a href="https://colab.research.google.com/github/sv650s/amazon-review-classification/blob/master/notebooks/deep_learning/6.5.6-LSTMB16-GloVe-problematic-categories-all-ratings-1m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will look at review samples from mis-classified examples

We are loading pre-trained model with 1mil example dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
DRIVE_DIR = "drive/My Drive/Springboard/capstone"
sys.path.append(DRIVE_DIR)


%tensorflow_version 2.x


import tensorflow as tf
# checl to make sure we are using GPU here
tf.test.gpu_device_name()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TensorFlow 2.x selected.


'/device:GPU:0'

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, \
    SpatialDropout1D, Flatten, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.initializers import Constant


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight


import pandas as pd
import numpy as np
from IPython.display import SVG
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
import logging


import util.dict_util as du
import util.plot_util as pu
import util.file_util as fu
import util.keras_util as ku
import util.report_util as ru

import random

# fix random seeds
tf.compat.v1.reset_default_graph()
tf.compat.v1.set_random_seed(1)
random.seed(1)
np.random.seed(1)

logging.basicConfig(level=logging.ERROR)

%matplotlib inline
sns.set()


DATE_FORMAT = '%Y-%m-%d'
TIME_FORMAT = '%Y-%m-%d %H:%M:%S'
LABEL_COLUMN = "star_rating"
REVIEW_COLUMN = "review_body"


DEBUG = False


LSTM_DIM = 16 # total LSTM units
MODEL_NAME = f"LSTMB{LSTM_DIM}"
ARCHITECTURE = f"1x{LSTM_DIM}"
DESCRIPTION = f"1 Layer {LSTM_DIM} LSTM Units, No Dropout, GloVe Embedding, Balanced Weights"
FEATURE_SET_NAME = "glove"
PATIENCE = 4

SAMPLES = "1m"

if DEBUG:
  DATA_FILE = f'{DRIVE_DIR}/data/amazon_reviews_us_Wireless_v1_00-test-preprocessed.csv'
  MODEL_NAME = f'test-{MODEL_NAME}'
  MISSING_WORDS_FILE = f'{DRIVE_DIR}/reports/glove_embedding-missing_words-test.csv'
  ku.ModelWrapper.set_report_filename('test-dl_prototype-report.csv')
else:
  DATA_FILE = f"{DRIVE_DIR}/data/amazon_reviews_us_Wireless_v1_00-{SAMPLES}-preprocessed.csv"
  MISSING_WORDS_FILE = f'{DRIVE_DIR}/reports/glove_embedding-missing_words-{SAMPLES}.csv'
  ku.ModelWrapper.set_report_filename('glove_embedding-dl_prototype-report.csv')


EMBEDDING_FILE = f'{DRIVE_DIR}/data/embeddings/glove.840B.300d.txt'

# first layer filter
FILTER1 = 32
# Network Settings
KERNEL_SIZE=3



# length of our embedding - 300 is standard
EMBED_SIZE = 300
EPOCHS  = 50
BATCH_SIZE = 128

# From EDA, we know that 90% of review bodies have 100 words or less, 
# we will use this as our sequence length
MAX_SEQUENCE_LENGTH = 100


pd.set_option("max_colwidth", 200)

In [3]:
# Load Report file
report = pd.read_csv(f'{DRIVE_DIR}/reports/glove_embedding-dl_prototype-report.csv', quotechar="'")
report = report[(report.model_name == 'LSTMB16') & (report.train_examples == 746766)]
report

,accuracy,architecture,batch_size,class_weight,classification_report,confusion_matrix,description,embedding,epochs,evaluate_time_min,feature_set_name,file,loss,max_sequence_length,model_file,model_json_file,model_name,predict_time_min,roc_auc,sampling_type,status,status_date,test_examples,test_features,tokenizer_file,train_examples,train_features,train_time_min,weights_file,network_history_file
11,0.707173,1x16,128.0,"[1.4228484463085092, 3.0270973626206583, 2.2103804999334016, 1.200130175375158, 0.3729699357961058]","{""1"": {""precision"": 0.6649026010584393, ""recall"": 0.8396610073088189, ""f1-score"": 0.7421325155841544, ""support"": 35163}, ""2"": {""precision"": 0.38207637689502977, ""recall"": 0.12068129470238817, ""f1-...","[[29525, 1480, 1969, 398, 1791], [8086, 1991, 3933, 837, 1651], [4024, 1369, 7685, 4346, 4935], [1236, 248, 3622, 12214, 23995], [1534, 123, 1389, 5925, 124616]]","1 Layer 16 LSTM Units, No Dropout, GloVe Embedding, Balanced Weights",300.0,23.0,0.69,glove,drive/My Drive/Springboard/capstone/data/amazon_reviews_us_Wireless_v1_00-1m-preprocessed.csv,0.74922,100.0,drive/My Drive/Springboard/capstone/models/LSTMB16-1x16-glove-sampling_none-995688-100-star_rating-model.h5,drive/My Drive/Springboard/capstone/models/LSTMB16-1x16-glove-sampling_none-995688-100-star_rating-model.json,LSTMB16,0.53,"{""auc_1"": 0.9631105248092278, ""auc_2"": 0.891297088228741, ""auc_3"": 0.8619117785493009, ""auc_4"": 0.7866581460860135, ""auc_5"": 0.9099328736144383, ""auc_micro"": 0.9281704654129673, ""auc_macro"": 0.882...",none,success,2020-02-15 01:27:05,248922.0,100.0,drive/My Drive/Springboard/capstone/models/LSTMB16-1x16-glove-sampling_none-995688-100-star_rating-tokenizer.pkl,746766.0,100.0,24.46,drive/My Drive/Springboard/capstone/models/LSTMB16-1x16-glove-sampling_none-995688-100-star_rating-weights.h5,drive/My Drive/Springboard/capstone/reports/LSTMB16-1x16-glove-sampling_none-995688-100-star_rating-history.pkl


In [4]:
DATA_FILE = report.file.values[0]
print(f'Reading datafile: {DATA_FILE}')
df = pd.read_csv(DATA_FILE, encoding='utf8', engine='python')

rating = df[LABEL_COLUMN]
reviews = df[REVIEW_COLUMN]

Reading datafile: drive/My Drive/Springboard/capstone/data/amazon_reviews_us_Wireless_v1_00-1m-preprocessed.csv


# Preprocessing

In [5]:


# pre-process our lables
# one hot encode our star ratings since Keras/TF requires this for the labels
ohe = OneHotEncoder()
y = ohe.fit_transform(rating.values.reshape(len(rating), 1)).toarray()


# split our data into train and test sets
reviews_train, reviews_test, y_train, y_test = train_test_split(reviews, y, random_state=1)

with open(report.tokenizer_file.values[0], 'rb') as file:
  t = pickle.load(file)

# Pre-process our features (review body)
# t = Tokenizer(oov_token="<UNK>")
# fit the tokenizer on the documents
# t.fit_on_texts(reviews_train)
# tokenize both our training and test data
train_sequences = t.texts_to_sequences(reviews_train)
test_sequences = t.texts_to_sequences(reviews_test)

print("Vocabulary size={}".format(len(t.word_counts)))
print("Number of Documents={}".format(t.document_count))


# pad our reviews to the max sequence length
X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Train review vectors shape:', X_train.shape, ' Test review vectors shape:', X_test.shape)


Vocabulary size=109547
Number of Documents=746766
Train review vectors shape: (746766, 100)  Test review vectors shape: (248922, 100)


# Load Our Pre-trained Model

In [0]:
from tensorflow.keras.models import load_model

model = load_model(report.model_file.values[0])

Double checking our accuracy. It should be: 70.60%

In [7]:
scores = model.evaluate(X_test, y_test, verbose=1)


248922/248922 [==============================] - 114s 458us/sample - loss: 0.7492 - categorical_accuracy: 0.7072


In [8]:
print("Loss: %.2f%%" % (scores[0]*100))
print("Accuracy: %.2f%%" % (scores[1]*100))


Loss: 74.92%
Accuracy: 70.72%


In [9]:
# look at a couple prediction results
y_predict = model.predict(X_test)
y_predict[:5]

array([[6.6044432e-04, 1.2262755e-03, 1.0042071e-02, 1.4336163e-01,
        8.4470958e-01],
       [4.8195251e-04, 7.5134105e-04, 8.3812112e-03, 1.2789713e-01,
        8.6248833e-01],
       [3.7696629e-04, 1.3154644e-03, 1.9974884e-02, 1.9462089e-01,
        7.8371179e-01],
       [2.0780079e-03, 3.1067615e-03, 1.8840974e-02, 1.2706077e-01,
        8.4891349e-01],
       [9.5476207e-05, 1.0342833e-04, 1.2387980e-03, 5.4122183e-02,
        9.4444013e-01]], dtype=float32)

# Unencode our labels back to 1 to 5 so we can look at confusion matrix and classification report

In [0]:
y_test_unencoded = ku.unencode(y_test)
y_predict_unencoded = ku.unencode(y_predict)

In [11]:
from sklearn.metrics import confusion_matrix, classification_report

cr = classification_report(y_test_unencoded, y_predict_unencoded)
print(cr)

              precision    recall  f1-score   support

           1       0.66      0.84      0.74     35163
           2       0.38      0.12      0.18     16498
           3       0.41      0.34      0.38     22359
           4       0.51      0.30      0.38     41315
           5       0.79      0.93      0.86    133587

    accuracy                           0.71    248922
   macro avg       0.55      0.51      0.51    248922
weighted avg       0.67      0.71      0.67    248922



In [12]:
confusion_matrix = confusion_matrix(y_test_unencoded, y_predict_unencoded)
print(confusion_matrix)

[[ 29525   1480   1969    398   1791]
 [  8086   1991   3933    837   1651]
 [  4024   1369   7685   4346   4935]
 [  1236    248   3622  12214  23995]
 [  1534    123   1389   5925 124616]]


In [13]:
classification_report = classification_report(y_test_unencoded, y_predict_unencoded)
print(classification_report)

              precision    recall  f1-score   support

           1       0.66      0.84      0.74     35163
           2       0.38      0.12      0.18     16498
           3       0.41      0.34      0.38     22359
           4       0.51      0.30      0.38     41315
           5       0.79      0.93      0.86    133587

    accuracy                           0.71    248922
   macro avg       0.55      0.51      0.51    248922
weighted avg       0.67      0.71      0.67    248922



In [14]:
result = pd.DataFrame({"test": y_test_unencoded, "predict": y_predict_unencoded})
result.head()

,test,predict
0,5,5
1,5,5
2,5,5
3,5,5
4,5,5


In [15]:
# add column to tell use which ones are misclassified
result["correct"] = result.test == result.predict

print(result[result.correct == True].sample(5))

result[result.correct == False].sample(5)

        test  predict  correct
190112     5        5     True
127893     5        5     True
94518      5        5     True
240572     5        5     True
40951      5        5     True


,test,predict,correct
246527,2,3,False
150148,2,1,False
33492,3,2,False
179340,4,5,False
107012,5,4,False


# Load our embedding indexer to see what words may have been removed

In [16]:
EMBEDDING_INDEX_FILE = f'{DRIVE_DIR}/models/glove.840B.300d-embedding_index'

embeddings_index = {}

if os.path.exists(f'{EMBEDDING_INDEX_FILE}.npy'):
  print(f'Loading {EMBEDDING_INDEX_FILE}.npy')
  embeddings_index = np.load(f'{EMBEDDING_INDEX_FILE}.npy', 
                             allow_pickle = True).item()
else:
  print('Indexing word vectors.')

  with open(EMBEDDING_FILE) as f:
      for line in f:
          word, coefs = line.split(maxsplit=1)
          coefs = np.fromstring(coefs, 'f', sep=' ')
          embeddings_index[word] = coefs
  np.save(EMBEDDING_INDEX_FILE, embeddings_index)

print(type(embeddings_index))
print(np.shape(embeddings_index))
print('Found %s word vectors.' % len(embeddings_index))

Loading drive/My Drive/Springboard/capstone/models/glove.840B.300d-embedding_index.npy
<class 'dict'>
()
Found 2195884 word vectors.


# reset reviews index so it matches our result dataframe


In [0]:
reviews_reset = reviews_test.reset_index().rename({"index": "index_orig"}, axis=1)

# Incorrectly classified 1-star reviews

In [18]:
incorrect_result = result[(result.test == 1) & (result.correct == False)]
incorrect_idx = incorrect_result.index.tolist()
incorrect_pd = pd.merge(incorrect_result, reviews_reset.loc[incorrect_idx], left_index = True, right_index = True).sample(min(20, len(incorrect_result)), random_state=1)
incorrect_pd["out_of_vocab"] = ""

for index, row in incorrect_pd.iterrows():
  review_body = row.review_body
  missing_words = [word for word in review_body.split() if word not in embeddings_index.keys()]
  incorrect_pd.loc[index, "out_of_vocab"] = ", ".join(missing_words)

incorrect_pd

,test,predict,correct,index_orig,review_body,out_of_vocab
78152,1,3,False,721083,case not rubber would expect very rigid showed sign wear couple week use not hard phone case either,
84008,1,5,False,645989,becarefull guy selling used battry instead new one cell phone turning decided buy new battery ordered one two day problem still cellphone almost going buy new cell phone decide buy another battry ...,
41477,1,5,False,667631,work perfect price good free freight gone many retail store none no idea find,
90324,1,5,False,374697,power recommended use cell phone will quickly distort phone fry battery,
41459,1,5,False,184562,sound quality not very good beginning quit taking charge 1 5 month changed usb cord power source no avail advise staying away item junk revised changing rating 1 4 star possibly best customer serv...,
75727,1,5,False,552124,seemed like great idea no fixed device per port initial result fantastic would happily charge android phone ipads android tablet iphones without problem port stopped working 2 week older anker fix...,
142551,1,3,False,873249,said haha keyboard small able touch type may work not button work though paired easily google nexus 7,
152371,1,2,False,416528,pro arrived ti ame look feel exactly like original fit snugly phone slotcons way sensitive button slightly sunk turned piece plastic supporting button broken no quality check prior shipping unlike...,slotcons
206812,1,3,False,646553,privacy screen did not stick phone without bubble appearing place did everything instruction said do,
17015,1,2,False,536439,disappointed mic did not standard function normal apple mic volume week putting pocket front cover mic fell also size plastic housing somewhat irritating ear,


# Incorrectly classified 2-star ratings

In [19]:
incorrect_result = result[(result.test == 2) & (result.correct == False)]


incorrect_idx = incorrect_result.index.tolist()
incorrect_pd = pd.merge(incorrect_result, reviews_reset.loc[incorrect_idx], left_index = True, right_index = True).sample(min(20, len(incorrect_result)), random_state=1)
incorrect_pd["out_of_vocab"] = ""

for index, row in incorrect_pd.iterrows():
  review_body = row.review_body
  missing_words = [word for word in review_body.split() if word not in embeddings_index.keys()]
  incorrect_pd.loc[index, "out_of_vocab"] = ", ".join(missing_words)

incorrect_pd

,test,predict,correct,index_orig,review_body,out_of_vocab
197981,2,1,False,361502,did not last 6 month skin peeled,
79866,2,1,False,586353,worked great 2 month 1 1 2 month light would not come ti ame still worked 2 month say charging will not actually charge phone,
214594,2,1,False,768101,doe not charge charge doe not work ever since came try ti charge phone doe not charge,
45690,2,1,False,896131,warning head cable not exact size shape stock apple cable will not fit phone use bumper case fine removing bumper case every ti ame wish charge phone go ahead otherwise buy different product,
182039,2,4,False,694058,doe job do not charge battery best full battery watching movie,
34548,2,3,False,60738,ok not great click pause u change station connect ipod look like short turn back using usb cord tends also playing station decides not play sound change station return back hear station want doe l...,
194589,2,5,False,591534,selection product based pri amarily color amount natural color iphone visible still providing protection edge iphone,amarily
58808,2,1,False,791493,cheap product know say get pay everything fall including bow service mail price cant bi,
28147,2,1,False,303553,not waterproof advertised,
210006,2,1,False,506148,didnt fit well,


# Incorrectly classified 3-star ratings

In [20]:
incorrect_result = result[(result.test == 3) & (result.correct == False)]

incorrect_idx = incorrect_result.index.tolist()
incorrect_pd = pd.merge(incorrect_result, reviews_reset.loc[incorrect_idx], left_index = True, right_index = True).sample(min(20, len(incorrect_result)), random_state=1)
incorrect_pd["out_of_vocab"] = ""

for index, row in incorrect_pd.iterrows():
  review_body = row.review_body
  missing_words = [word for word in review_body.split() if word not in embeddings_index.keys()]
  incorrect_pd.loc[index, "out_of_vocab"] = ", ".join(missing_words)

incorrect_pd

,test,predict,correct,index_orig,review_body,out_of_vocab
16952,3,1,False,323557,phone still didnt work new battery phone died,
57253,3,4,False,957837,good product price magellan product past like ease navigation doe not bell wistles higher prioed model will suit purpose,prioed
193140,3,4,False,584285,great look good protection side volume button very hard press press fingernail much hard,
40630,3,1,False,985693,received unit christmas purchased directly delphi lock can not find rhyme reason locking seems random can not turn lock wait battery drain usually around 2 5 hour really upset called delphi said s...,
88139,3,1,False,220375,doe not charge fast authentic htc one cable provided phone,
183607,3,5,False,954251,seidio case droid recently found otterbox started making one two previous otterbox case blackberry curve pearl knew strong really big fan however one disappointed front piece cracked second day di...,
213024,3,5,False,126313,needed mount small specialized computer new race car worked okay needed reinforcing handle weight,
247535,3,2,False,936411,very thin fli amsy case front broke week bought allegedly identical case though black front part fine unpredictable,amsy
21895,3,5,False,116848,good,
99130,3,1,False,201890,poor fit compartment kept popping used two season died purchased new one recently different vendor little capacity perfect fit,


# 5-star misclassifed as 1-star

In [21]:
incorrect_result = result[(result.test == 5) & (result.predict == 1) & (result.correct == False)]

incorrect_idx = incorrect_result.index.tolist()
incorrect_pd = pd.merge(incorrect_result, reviews_reset.loc[incorrect_idx], left_index = True, right_index = True).sample(min(20, len(incorrect_result)), random_state=1)
incorrect_pd["out_of_vocab"] = ""

for index, row in incorrect_pd.iterrows():
  review_body = row.review_body
  missing_words = [word for word in review_body.split() if word not in embeddings_index.keys()]
  incorrect_pd.loc[index, "out_of_vocab"] = ", ".join(missing_words)

incorrect_pd


,test,predict,correct,index_orig,review_body,out_of_vocab
11683,5,1,False,787088,way case could better would paid take case price low almost gave experience ipod touch 5 slippery day took get case constantly feared going drop moment put case fear went away blue ipod case allow...,
168067,5,1,False,970344,piece junk broke 3 ti ames will not fix not good model cheaply built spend money get expensive one did,
43488,5,1,False,832208,charging unit seemed work well unfortunately battery did not thanks sending wish could used,
20315,5,1,False,775982,far no issue love little device latest trend seems toward making smartphones no longer allow user switch phone battery make absolutely no sense suppose force replace update entire phone three year...,
115825,5,1,False,703847,samsung view cover did not secure screen pretty sure actually caused screen get scratched galaxy nexus actually managed crack corner glass thought should careful case everything wanted view functi...,
220582,5,1,False,413665,do not listen anyone doe not raton 3 case magnificent no flaw do not get lifeproof cause gone 2 phone got broken ran one car dropped ti ame count case definition raw,
111308,5,1,False,716282,case rated 5 look beautifuleasy snap offvery inexpensive could not amagine 2 item would wonderful literally no expectation received itthe e mail said item would take least 4 week deliver shipped c...,"beautifuleasy, offvery"
154294,5,1,False,420948,excellent product unlike review plug play instruction could better individual wire label could named ordinary guy very pleased others complained unit hanging mine work fine navigation slow powerin...,
28470,5,1,False,210274,no matter drop break,
181438,5,1,False,986263,product exactly wanted except one broken received much trouble return,


# 1-star misclassified as 5-star


In [22]:
incorrect_result = result[(result.test == 1) & (result.predict == 5) & (result.correct == False)]

incorrect_idx = incorrect_result.index.tolist()
incorrect_pd = pd.merge(incorrect_result, reviews_reset.loc[incorrect_idx], left_index = True, right_index = True).sample(min(20, len(incorrect_result)), random_state=1)
incorrect_pd["out_of_vocab"] = ""

for index, row in incorrect_pd.iterrows():
  review_body = row.review_body
  missing_words = [word for word in review_body.split() if word not in embeddings_index.keys()]
  incorrect_pd.loc[index, "out_of_vocab"] = ", ".join(missing_words)

incorrect_pd

,test,predict,correct,index_orig,review_body,out_of_vocab
173577,1,5,False,994638,purchased used nextel assumption owned phone would not locked sort contract nextel wrong whether give free phone signing contract pay cash used phone must sign nextels contract nextel would not le...,amum
91042,1,5,False,471497,purchased unit replace older malfunctioning unit antenna combo sirius wanted charge switch new radio active account told already paying monthly fee no need charge change hardware using would not i...,
111674,1,5,False,503588,case sane exact case fosmon tpu case half price inc shipping despite review state case lip protect screen doe not lip placed ruler top phone lengthwise well width wise see absolutely no light rule...,fosmon
106040,1,5,False,645029,thought good buy cause bought 2 wire iphone work great 7 month not case one girl friend iphone 5 havent contacted company cause not sure will do thing late rei ambursed give new one amazon reading...,ambursed
215459,1,5,False,167195,bought bf love,
170903,1,5,False,941536,dont fooled great pricing item horrible worked exactly 2 day u rig electrical tape work,
27683,1,5,False,355687,one not one wanted wanted one flower even showed picture ordering,
125919,1,5,False,41000,charger not best ended buying well known brand night day difference quality charger hard ti ame charging phone case charger did work took extremely long ti ame charge phone spend extra dollar will...,
22857,1,5,False,576725,usually one buy newww product expect part original packaging however not case do not like product nice going amazon,
80067,1,5,False,918086,will work phone touch screen arent sensitive heat aka finger not like big fat stylus rubber end bought thought smaller version not sure work wonderfully type touch screen not one samsung captivate,


# 4-star misclassified as 5-star

In [23]:
incorrect_result = result[(result.test == 4) & (result.predict == 5) & (result.correct == False)]

incorrect_idx = incorrect_result.index.tolist()
incorrect_pd = pd.merge(incorrect_result, reviews_reset.loc[incorrect_idx], left_index = True, right_index = True).sample(min(20, len(incorrect_result)), random_state=1)
incorrect_pd["out_of_vocab"] = ""

for index, row in incorrect_pd.iterrows():
  review_body = row.review_body
  missing_words = [word for word in review_body.split() if word not in embeddings_index.keys()]
  incorrect_pd.loc[index, "out_of_vocab"] = ", ".join(missing_words)

incorrect_pd

,test,predict,correct,index_orig,review_body,out_of_vocab
222823,4,5,False,16738,like,
129937,4,5,False,817689,nice looking case very inexpensive seems fit very well excellent value feel good hand little softer like therefore likely prone losing shape tightness ti ame,
85088,4,5,False,343068,great quality easy place screen,
216959,4,5,False,656096,best bang buck 2 battery charger soon get home work swap battery,
14477,4,5,False,492729,used kit replace battery kindle fire hd 7 everything needed right size pleased affordable loved driver magnetized did not worry much loosing tiny screw easy get put back driver will last several p...,
3145,4,5,False,323341,work exactly expected convenient suckion cup need dashboard space,suckion
235826,4,5,False,231869,very good,
224670,4,5,False,5682,like bigger screen camera excellent seller got promised mahalo,
2247,4,5,False,766799,saw tv episode one favorite show would already bought itmade love much also get compli ament folk really look awesome doesnt add bulk feel pretty secure strongly recommend,itmade
241685,4,5,False,249852,great product deffently tough except end break easily using around house stop ani amals biting cord perfect,amals


In [24]:
print(datetime.now())

2020-02-15 02:22:06.078683
